# Capítulo 02: Instalação e Configuração do Iceberg

Este notebook detalha a instalação e configuração do DuckDB com Iceberg.

**Objetivo:** Dominar a configuração do ambiente e preparar diferentes cenários de uso.

**Conteúdo:**
- Instalação completa das dependências
- Verificação de instalação
- Configurações para desenvolvimento e produção
- Testes de conectividade

## 2. Verificação da Instalação Básica

## 9. Resumo

Neste capítulo você aprendeu:
- ✅ Como instalar e verificar a extensão Iceberg
- ✅ Configurar ambientes de desenvolvimento e produção
- ✅ Testar instalação e conectividade
- ✅ Criar configurações reutilizáveis

**Próximo:** Capítulo 03 - Leitura de Tabelas Iceberg

In [ ]:
class IcebergConfig:
    """Configuração padrão para DuckDB + Iceberg"""
    
    @staticmethod
    def setup(db_path=':memory:', config=None):
        """
        Configura DuckDB com Iceberg
        
        Args:
            db_path: Caminho do banco (default: in-memory)
            config: Dict com configurações adicionais
        """
        default_config = {
            'threads': 4,
            'memory_limit': '4GB'
        }
        
        if config:
            default_config.update(config)
        
        # Conectar
        con = duckdb.connect(db_path, config=default_config)
        
        # Instalar extensões
        for ext in ['iceberg', 'httpfs']:
            con.execute(f"INSTALL {ext}")
            con.execute(f"LOAD {ext}")
        
        return con

# Exemplo de uso
con_custom = IcebergConfig.setup(
    db_path='my_analytics.duckdb',
    config={'threads': 8, 'memory_limit': '8GB'}
)
print("✅ Configuração customizada carregada!")

## 8. Classe de Configuração Reutilizável

In [ ]:
import requests

def check_s3_connectivity():
    """Verifica conectividade com S3"""
    try:
        response = requests.get('https://s3.amazonaws.com', timeout=5)
        print(f"✅ S3 acessível (status: {response.status_code})")
        return True
    except Exception as e:
        print(f"❌ Problema de conectividade: {e}")
        print("   Verifique firewall/proxy")
        return False

# Testar conectividade
check_s3_connectivity()

## 7. Teste de Conectividade (S3/Cloud)

In [ ]:
def test_iceberg_basic():
    """Testa instalação básica do Iceberg"""
    con = duckdb.connect()
    
    try:
        safe_install_ext(con, "iceberg")
        print("✅ Extensão Iceberg funcionando corretamente")
        return True
    except Exception as e:
        print(f"❌ Erro ao carregar Iceberg: {e}")
        return False

# Executar teste
test_iceberg_basic()

## 6. Testes de Instalação

In [ ]:
# Configuração para ambiente de produção
con_prod = duckdb.connect('prod.duckdb')
safe_install_ext(con_prod, "iceberg")

# Extensões para cloud
con_prod.execute("INSTALL httpfs")
con_prod.execute("LOAD httpfs")

# Configurações para produção
con_prod.execute("SET threads = 8")
con_prod.execute("SET memory_limit = '16GB'")

print("✅ Ambiente de produção configurado!")
print(f"   - Threads: 8")
print(f"   - Memória: 16GB")
print(f"   ⚠️  NÃO habilitar flags 'unsafe' em produção!")

## 5. Configuração para Produção

Ambiente otimizado para cargas de trabalho reais.

In [ ]:
# Configuração para ambiente de desenvolvimento
con_dev = duckdb.connect('dev.duckdb')  # Banco persistente
safe_install_ext(con_dev, "iceberg")

# Configurações otimizadas para desenvolvimento
con_dev.execute("SET threads = 2")
con_dev.execute("SET memory_limit = '2GB'")

# Criar diretório para testes locais
os.makedirs('iceberg_tables', exist_ok=True)

print("✅ Ambiente de desenvolvimento configurado!")
print(f"   - Threads: 2")
print(f"   - Memória: 2GB")
print(f"   - Diretório: ./iceberg_tables")

## 4. Configuração para Desenvolvimento

Ambiente otimizado para testes locais.

In [ ]:
def setup_duckdb_iceberg():
    """Configura DuckDB com todas as extensões necessárias"""
    con = duckdb.connect()
    
    # Extensões para Iceberg + Cloud Storage
    extensions = ['iceberg', 'httpfs']  # httpfs para S3/HTTP
    
    for ext in extensions:
        print(f"Instalando {ext}...")
        safe_install_ext(con, ext)
    
    print("\n✅ Setup completo!")
    return con

# Executar setup
con = setup_duckdb_iceberg()

## 3. Setup Completo com Múltiplas Extensões

Para trabalhar com Iceberg em cloud (S3, Azure), instalamos extensões adicionais.

In [ ]:
# Criar conexão e instalar extensão Iceberg
con = duckdb.connect()
safe_install_ext(con, "iceberg")

# Verificar instalação
result = con.execute("""
    SELECT extension_name, loaded, installed
    FROM duckdb_extensions()
    WHERE extension_name = 'iceberg'
""").fetchone()

print(f"\nStatus da extensão Iceberg:")
print(f"  - Instalado: {result[2]}")
print(f"  - Carregado: {result[1]}")

In [ ]:
# Verificar versão do DuckDB
version = duckdb.__version__
print(f"DuckDB versão: {version}")

# Iceberg requer DuckDB >= 0.9.0
print("✅ Versão compatível com Iceberg" if version >= '0.9.0' else "⚠️  Atualize o DuckDB")

In [ ]:
# Importar bibliotecas e funções auxiliares
import duckdb
import os
import importlib.util


def has_module(name):
    """Verifica se módulo está disponível"""
    return importlib.util.find_spec(name) is not None


def safe_install_ext(con, ext_name):
    """Instala e carrega extensão com tratamento de erros"""
    try:
        con.execute(f"INSTALL {ext_name}")
        con.execute(f"LOAD {ext_name}")
        print(f"✅ {ext_name} instalado e carregado")
        return True
    except Exception as e:
        print(f"⚠️  Falha ao instalar {ext_name}: {e}")
        return False


print("Funções auxiliares carregadas!")

In [ ]:
# Instalar pacotes necessários
import sys
!{sys.executable} -m pip install -q duckdb pyarrow requests

## 1. Instalação das Dependências